# Pre-Pre-Processing

Before I start performing any EDA or modeling, I will pre-emptively split my data to have a separate training set of data.

Since the data consists of temporal data (hotel reservations across multiple years), I will split the data into two groups based on two datetime features: Booking Date and the Arrival date.

---

# Consulting ChatGPT

I asked ChatGPT to help me decide if one feature would be better than the other. It's responses are as follows:

## Booking Date

### Pros:
> * *Forecasting Lead Time:* Splitting by booking date allows you to model the lead time between booking and arrival, which is valuable for forecasting future demand well in advance.
> * *Real-world Scenario Modeling:* This approach mirrors real-world scenarios more closely, where predictions are made as bookings occur, regardless of the future arrival date.
> * *Dynamic Pricing and Revenue Management:* Particularly useful for applications in dynamic pricing and revenue management, where understanding booking patterns ahead of time can inform strategic decisions.

### Cons:
> * *Increased Complexity:* The time gap between booking and arrival dates introduces additional complexity, as you need to account for changes over time that could affect the stay (e.g., cancellations, modifications).



## Arrival Date

### Pros:
> * *Cleaner Data Relationships:* Utilizing the arrival date might result in cleaner, more straightforward modeling, as it directly ties the prediction to the period of service or product consumption.
> * *Seasonality and Trend Analysis:* Easier to model seasonal trends and effects, as the arrival date directly corresponds to the period being analyzed.
> * *Accuracy in Performance Metrics:* Predictions based on arrival date can be more closely aligned with actual occupancy and revenue, potentially improving model accuracy in terms of performance metrics.

### Cons:

> * *Reduced Forecasting Horizon:* The model may be less effective at predicting bookings well in advance since it's oriented around the arrival date. This could limit its usefulness for long-term planning.
> * *Possible Lag in Actionable Insights:* May not provide as much lead time for implementing strategies based on the predictions, such as staffing or promotional offers, since the focus is on the period closer to the actual stay.

---

# Consultation Conclusions

After considering ChatGPT's suggestions and insights, I will take a more greedy approach and create separate datasets for both the booking and arrival dates. This will give me more flexibility when modeling as I will have different time perspectives to utilize for different purposes (e.g., future forecasting vs. analyzing actualized performance).

---

# Date Preparation

Before I can split the datasets, I need to perform some slight feature engineering. The source datasets do not have an exact datetime feature for the arrival date, only for the booking date. I will use the separate Year, Month, and Day of Month features to create an `Arrival_Date` feature, then use this feature for splitting my data.

In [1]:
import pandas as pd

In [2]:
## Sharing 'df_data' to reuse code from a prior notebook
path1 = './data/H1.parquet'
df_data = pd.read_parquet(path1)

# ## Sharing 'df_data' to reuse code from a prior notebook
# path2 = './data/H2.parquet'
# df_data = pd.read_parquet(path2)

In [ ]:
## Convert Arrival columns to strings

arrival_date_cols = ['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth']

arrival_date_cols_str = df_data[arrival_date_cols].astype(str)
arrival_date_cols_str.head()

In [14]:
import pandas as pd
import glob
import os

# Define the directory containing the files
directory = "C:\\Users\\Ben\\OneDrive\\Documents\\Data Science Notebooks and Code\\SBN_Notebooks\\Hotel_Cancel_Culture\\data\\Supplemental"

# Define the file patterns to search for
file_patterns = ['*.xls', '*.xlsx', '*.csv', '*.tsv', '*.txt']

# Generate a list of all files matching the patterns
files = []
for pattern in file_patterns:
    files.extend(glob.glob(os.path.join(directory, pattern)))

# Function to determine the separator based on file extension
def get_separator(file_name):
    if file_name.endswith('.csv'):
        return ','
    elif file_name.endswith('.tsv') or file_name.endswith('.txt'):
        return '\t'
    else:
        return None  # For Excel files, the separator is not needed

# Loop through each file, read it into a DataFrame, save it as Parquet, and remove the original file
for file_path in files:
    separator = get_separator(file_path)
    
    # Read the file based on its extension
    if file_path.endswith('.xls') or file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        df = pd.read_csv(file_path, sep=separator)
    
    # Define the output file path
    parquet_path = os.path.splitext(file_path)[0] + '.parquet'
    
    # Convert and save the DataFrame as Parquet
    df.to_parquet(parquet_path, engine='pyarrow', compression='brotli')
    print(f'Converted {file_path} to {parquet_path} with brotli compression.')
    
    # Remove the original file
    os.remove(file_path)
    print(f'Removed original file: {file_path}')

C:\Users\Ben\anaconda3\envs\dojo-env\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Converted C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\Annual_Total_Hotels_PRT.xlsx to C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\Annual_Total_Hotels_PRT.parquet with brotli compression.
Removed original file: C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\Annual_Total_Hotels_PRT.xlsx
Converted C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\climate-change_prt.xlsx to C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\climate-change_prt.parquet with brotli compression.
Removed original file: C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\climate-change_prt.xlsx
Converted C

C:\Users\Ben\anaconda3\envs\dojo-env\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Ben\anaconda3\envs\dojo-env\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Ben\anaconda3\envs\dojo-env\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Converted C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\trade_prt.xlsx to C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\trade_prt.parquet with brotli compression.
Removed original file: C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\trade_prt.xlsx
Converted C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\urban-development_prt.xlsx to C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\urban-development_prt.parquet with brotli compression.
Removed original file: C:\Users\Ben\OneDrive\Documents\Data Science Notebooks and Code\SBN_Notebooks\Hotel_Cancel_Culture\data\Supplemental\urban-development_prt.xlsx
Converted C:\Users\Ben\OneDrive\Documents\Da

In [ ]:
import fitz  # PyMuPDF

def compress_pdf(input_path, output_path):
    doc = fitz.open(input_path)
    # Save the document with compression options
    doc.save(output_path, garbage=4, deflate=True)
    doc.close()
    print(f"Compressed PDF saved to {output_path}")

# Specify the input and output file paths
input_pdf_path = "C:\\Users\\Ben\\OneDrive\\Documents\\Data Science Notebooks and Code\\SBN_Notebooks\\Hotel_Cancel_Culture\\data\\Supplemental\\JOIN_Benchmarking_Tool.pdf"
output_pdf_path = "C:\\Users\\Ben\\OneDrive\\Documents\\Data Science Notebooks and Code\\SBN_Notebooks\\Hotel_Cancel_Culture\\data\\Supplemental\\JOIN_Benchmarking_Tool_Compressed.pdf"

compress_pdf(input_pdf_path, output_pdf_path)
